In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, matthews_corrcoef, recall_score, precision_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.metrics import roc_curve
import torch.nn.functional as F

In [ ]:
loaded_datasets_info = torch.load('/root/autodl-tmp/data/saved_datasets.pth', weights_only=False)
train_dataset = loaded_datasets_info['train_dataset']
val_dataset = loaded_datasets_info['val_dataset']

In [ ]:
batch_size = 10
loaded_train_dataset = DataLoader(train_dataset, batch_size = batch_size, shuffle = False)
loaded_val_dataset = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

In [ ]:
size = 8
class Net_conv(torch.nn.Module):
    def __init__(self, input_length):
        super(Net_conv, self).__init__()
        self.block_1 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=1,
                            out_channels=size,
                            kernel_size=1,
                            stride=1,
                            padding=0),
            torch.nn.BatchNorm1d(size),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv1d(in_channels=size,
                                out_channels=2*size,
                                kernel_size=3,
                                stride=1,
                                padding=1),
            torch.nn.BatchNorm1d(2*size)
        )
        self.block_2 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=2*size,
                            out_channels=4*size,
                            kernel_size=1,
                            stride=1,
                            padding=0),
            torch.nn.BatchNorm1d(4*size),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv1d(in_channels=4*size,
                                out_channels=2*size,
                                kernel_size=3, 
                                stride=1,
                                padding=1),
            torch.nn.BatchNorm1d(2*size)
        )
        iutput_size_block_1 = (input_length - 1 + 2 * 0) // 1 + 1  
        output_size_block_2 = (iutput_size_block_1 - 1 + 2 * 0) // 1 + 1  
        num_channels_last_layer = 2*size 
        linear_input_size = num_channels_last_layer * output_size_block_2    
        self.linear_1 = torch.nn.Linear(linear_input_size, 3)
    
    def forward(self, x):
        shortcut = x.float()
        x = self.block_1(x)
        x = torch.nn.functional.relu(x + shortcut)    
        shortcut = x
        x = self.block_2(x)
        x = torch.nn.functional.relu(x + shortcut)     
        x = x.view(x.size(0), -1)
        x =  self.linear_1(x)
        return x

In [ ]:
device = "cuda"
model = Net_conv(input_length = 31).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  
criterion = nn.CrossEntropyLoss().to(device)
num_epochs = 100

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_indx, (inputs, labels) in enumerate(loaded_train_dataset):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    # Print average loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / (len(loaded_train_dataset) / batch_size)}")

In [ ]:
torch.save(model.state_dict(), '/root/autodl-tmp/model_params/CNN.pth')

In [ ]:
model.load_state_dict(torch.load('/root/autodl-tmp/model_params/CNN.pth', weights_only=False))

### Metrics function definition

In [ ]:
# get roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC
def calculate_multiclass_metrics(true_labels, predicted_labels, predicted_probabilities, num_classes):
    accuracy = accuracy_score(true_labels, predicted_labels)
    mcc = matthews_corrcoef(true_labels, predicted_labels)
    
    sensitivity_per_class = []
    specificity_per_class = []
    auc_per_class = []
    f1_per_class = []

    for i in range(num_classes):
        true_binary = (np.array(true_labels) == i).astype(int)
        pred_binary = (np.array(predicted_labels) == i).astype(int)

        cm = confusion_matrix(true_binary, pred_binary, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()

        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity_per_class.append(sensitivity)
        specificity_per_class.append(specificity)

        
        auc = roc_auc_score(true_binary, predicted_probabilities[:, i]) if len(np.unique(true_binary)) > 1 else 0
        auc_per_class.append(auc)

        f1 = f1_score(true_binary, pred_binary) if len(np.unique(true_binary)) > 1 else 0
        f1_per_class.append(f1)

    avg_sensitivity = np.mean(sensitivity_per_class)
    avg_specificity = np.mean(specificity_per_class)
    avg_auc = np.mean(auc_per_class) if auc_per_class else 0
    avg_f1 = np.mean(f1_per_class)

    print(f"Average AUC: {avg_auc:.4f}")
    print(f"Average Sensitivity: {avg_sensitivity:.4f}")
    print(f"Average Specificity: {avg_specificity:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Average F1-score: {avg_f1:.4f}")
    print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

### Training data metrics

In [ ]:
predicted_probabilities = []
true_labels = []
with torch.set_grad_enabled(False):
    for batch_indx, (inputs, labels) in enumerate(loaded_train_dataset):
        inputs = inputs.to(device)
        labels = labels.to(device)      
        outputs = model(inputs)
        predicted_probabilities.extend(outputs.tolist())
        true_labels.extend(labels.tolist())

In [ ]:
true_labels_ = np.argmax(true_labels, axis=-1)
predicted_labels = np.argmax(predicted_probabilities, axis=-1)
preds = torch.tensor(predicted_probabilities)
preds = F.softmax(preds, dim=-1)

train_metrics = calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=3)

In [ ]:
np.save('/root/autodl-tmp/ROC/CNN/y_train_pred.npy', preds)
np.save('/root/autodl-tmp/ROC/CNN/y_train.npy', true_labels)

### Validation data metrics

In [ ]:
predicted_probabilities = []  
true_labels = []  
with torch.set_grad_enabled(False): 
    for batch_indx, (inputs, labels) in enumerate(loaded_val_dataset):
        inputs = inputs.to(device)
        labels = labels.to(device)    
        outputs = model(inputs)
        predicted_probabilities.extend(outputs.tolist())
        true_labels.extend(labels.tolist())

In [ ]:
true_labels_ = np.argmax(true_labels, axis=-1)
predicted_labels = np.argmax(predicted_probabilities, axis=-1)
preds = torch.tensor(predicted_probabilities)
preds = F.softmax(preds, dim=-1)

test_metrics = calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=3)

In [ ]:
np.save('/root/autodl-tmp/ROC/CNN/y_val_pred.npy', preds)
np.save('/root/autodl-tmp/ROC/CNN/y_val.npy', true_labels)